# The version includes feature extraction

# How to use:
set `TRAIN_OR_TEST` variable to `TRAIN_OR_TEST = 'train'` to generate features for the train images and `TRAIN_OR_TEST = 'test'` for test images
you can specify own backbone model via `model` and `layer` parameters


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
import pandas as pd

import os
from PIL import Image
import gc
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import cohen_kappa_score, make_scorer
from tqdm import tqdm, tqdm_notebook

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
#import torchsummary
from PIL import Image
# Any results you write to the current directory are saved as output.

In [ ]:
INPUT_PATH = "../input/petfinder-adoption-prediction/"
TRAIN_OR_TEST = 'test'

train = pd.read_csv(INPUT_PATH+TRAIN_OR_TEST+'/'+TRAIN_OR_TEST+'.csv') # pd.read_csv(INPUT_PATH+'train/train.csv')
train_image = pd.DataFrame(os.listdir(INPUT_PATH+TRAIN_OR_TEST+'_images/')) # pd.DataFrame(os.listdir(INPUT_PATH+'train_images/'))

image_path = {"fnames":[],"id":[], "pic_id_int":[]}
for i in os.listdir(INPUT_PATH+TRAIN_OR_TEST+'_images/'): # os.listdir(INPUT_PATH+'train_images/'):
    image_path["fnames"].append(i) 
    image_path["id"].append(i.split("-")[0])
    image_path["pic_id_int"].append(int(i.split("-")[1].split(".jpg")[0]))
    
image_df = pd.DataFrame(image_path)
if TRAIN_OR_TEST == 'train':
    image_df = image_df.merge(right=train[["PetID", "AdoptionSpeed"]], left_on="id", right_on="PetID", how="left")
elif TRAIN_OR_TEST == 'test':
    image_df = image_df.merge(right=train[["PetID"]], left_on="id", right_on="PetID", how="left")
else:
    print("TRAIN_OR_TEST is incorrect")

image_df = image_df.drop(["id"], axis=1)
image_df.head(3)

In [ ]:
image_df_first_img = image_df[image_df.pic_id_int == 1]
print(image_df_first_img.shape[0], "first images")

In [ ]:
# we can load nets without internet:
# https://github.com/KaiyangZhou/deep-person-reid/issues/23
# https://www.kaggle.com/rohitgr/fastaiv1-with-densenet121-and-focal-loss/notebook
#learn.load("densenet121-a639ec97")

#### load model

In [ ]:
model = models.resnet101(pretrained=True)
#model = models.densenet121(pretrained=True)
#model61 = models.densenet161(pretrained=True)
#model101 = models.resnet101(pretrained=True)
#model151 = models.resnet152(pretrained=True)
#modelvgg19 = models.vgg19_bn(pretrained=True)

#### choose layer

In [ ]:
# you can check output size with torchsummary
# torchsummary.summary(model, (3, 224, 224))
# or layer = model._modules.get('avgpool')
layer = model.features.norm5
#layer = model.features.norm5
#layer161 = model61.features.norm5 # [-1, 2208, 7, 7]           4,416
#layer101 = model101.avgpool # [-1, 2048, 1, 1]
#layer151 = model151.avgpool # [-1, 2048, 1, 1]
#layervgg19 = modelvgg19.avgpool # [-1, 512, 7, 7]

In [ ]:
_ = model.eval()
'''
_ = model61.eval()
_ = model101.eval()
_ = model151.eval()
_ = modelvgg19.eval()
'''

#### Scale and normalize before a network

In [ ]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

#### get_vectors() - extract features from a minibatch

In [ ]:
def get_3d_image(img, image_name):
    raw_data = np.array(img)
    #print("Raw data shape:", raw_data.shape)
    if len(raw_data.shape) == 3 and raw_data.shape[2] == 3:
        return img
    elif len(raw_data.shape) == 3 and raw_data.shape[2] == 1:
        print("get_3d_image, case 2:", image_name)
        return Image.fromarray(np.stack((img.squeeze(),)*3, axis=-1), 'RGB')
    elif len(raw_data.shape) == 2:
        print("get_3d_image, case 3:", image_name)
        return Image.fromarray(np.stack((img,)*3, axis=-1), 'RGB')
    else:
        print("error in shape:", raw_data.shape)
        return -1

my_embedding = []
def get_vectors(image_names, outp_operation ):
    global my_embedding
    t_imges = []
    for image_name in image_names:
        # 1. Load the image with Pillow library
        img = Image.open(image_name)
        max_size = max(img.height, img.width)
        h_delta = (max_size - img.height)//2
        w_delta = (max_size - img.width)//2
        padding = transforms.Pad((w_delta, h_delta))
        t_imges.append(normalize(to_tensor(scaler(padding(get_3d_image(img, image_name))))))
    t_imges = torch.stack(t_imges)
    
    def copy_data(module, input, output):
        my_embedding.append(output.data.squeeze())
    h = layer.register_forward_hook(copy_data)
    model(t_imges)
    h.remove()
    
    #print(t_imges.shape)
    #print(my_embedding[0].detach().numpy().shape)
    #print(outp_operation(my_embedding[0]).detach().numpy().shape)
    
    return outp_operation(my_embedding.pop()).detach().numpy()

#### test the above extractor

In [ ]:
avgPool = nn.AdaptiveAvgPool3d((512,1,1)) # from 32,7,7
pic_one = INPUT_PATH+'train_images/'+'00156db4a-5.jpg'
pic_two = INPUT_PATH+'train_images/'+'00156db4a-4.jpg'
pic_two3 = INPUT_PATH+'train_images/'+'00156db4a-3.jpg'
pic_two2 = INPUT_PATH+'train_images/'+'00156db4a-2.jpg'
pic_two1 = INPUT_PATH+'train_images/'+'00156db4a-1.jpg'

pics_vectors = get_vectors([pic_one, pic_two,pic_two3,pic_two2,pic_two1], avgPool)

cnt_new_features = np.prod(pics_vectors[0,:,:,:].shape)
print("Number of new features:", cnt_new_features)
new_features_names = ['feature' + str(num) for num in range(cnt_new_features)]

In [ ]:
X_img_names = INPUT_PATH + TRAIN_OR_TEST + '_images/' + np.array(image_df_first_img.fnames)
X_ids = np.array(image_df_first_img.PetID)
y = None
if TRAIN_OR_TEST == 'train':
    y = np.array(image_df_first_img.AdoptionSpeed)

#### simulation of the batch loading

In [ ]:
'''
batch_size = 100
X_features_train = get_vectors(X_img_names[:batch_size], avgPool).reshape(batch_size, cnt_new_features)
y_train = y[:batch_size]
'''

#### generate features
~ 2 min for 10 iterations = for 1000 examples

In [ ]:
# We cannot sent batch size 14k+ to the net. let's do batch size ~500 and create features in a loop
print("Number of images to generate fetures:", X_img_names.shape)
cnt_batchs = 300
if TRAIN_OR_TEST == 'train':
    cnt_batchs = 300
else:
    cnt_batchs = 60
batch_ends_ids = np.linspace(50, len(X_img_names), cnt_batchs, dtype=int)
last_end_id = 0
X_features = []
for i in tqdm_notebook(range(cnt_batchs)):
    batch_size = batch_ends_ids[i] - last_end_id
    X_features.append(get_vectors(X_img_names[last_end_id:batch_ends_ids[i]], avgPool).reshape(batch_size, cnt_new_features))
    last_end_id = batch_ends_ids[i]
print("Stack batches in the feature list")
X_features = np.vstack(X_features)

np.save('X_features_resnet101'+TRAIN_OR_TEST, X_features)
np.save('X_img_names_resnet101'+TRAIN_OR_TEST, X_img_names)
if TRAIN_OR_TEST == 'train':
    np.save('y_resnet101'+TRAIN_OR_TEST, y)

## Now, I am loading data from the outputs above

In [ ]:
X_features = np.load('./X_features_resnet101.npy')
X_img_names = np.load('./X_img_names_resnet101.npy')
y = np.load('./y_resnet101.npy')

#### get dummy labels from $y \in \{0,1,2,3,4\}$. WE DO NOT USE THE FUNCTIONS HERE

In [ ]:
# input example: get_hard_dummy([0,1,2,1])
def get_hard_dummy(y, cnt_labels=5):
    dummies = np.zeros((len(y), cnt_labels))
    for idx, label in zip(range(len(y)), y):
        dummies[idx, label] = 1.0
    return dummies

# for y=1 return [0.3, 1, 0.3, 0, 0] instead of [0, 1, 0, 0, 0]
def get_soft_dummy(y, soft_zero = 0.3, cnt_labels=5):
    dummies = np.zeros((len(y), cnt_labels))
    for idx, label in zip(range(len(y)), y):
        dummies[idx, label] = 1.0
        if label == 0:
            dummies[idx, 1] = soft_zero
        elif label == cnt_labels - 1:
            dummies[idx, cnt_labels - 2] = soft_zero
        else:
            dummies[idx, label - 1], dummies[idx, label + 1] = soft_zero, soft_zero
    return dummies

#### Kappa Score

In [ ]:
# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

#### LGBM on features

In [ ]:
if TRAIN_OR_TEST == 'train':
    kappa_scorer = make_scorer(quadratic_weighted_kappa)
    kf = model_selection.KFold(n_splits = 5, random_state = 42, shuffle = True)
    model = lgb.LGBMClassifier(n_jobs=-1, random_state=42, n_estimators=500, reg_alpha=1.3, reg_lambda=1.3)
    scores = model_selection.cross_val_score(model, X_features, y, cv=kf, scoring = kappa_scorer)
    scores